In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 11.8 MB/s eta 0:00:00
Enter your Groq API Key: ··········


In [3]:
question = "Ravi has 5 tennis balls. He buys 3 more can of tennis balls. Each can has 2 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Ravi has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (3 * 2 = 6).

So, Ravi initially had 5 tennis balls. He bought 6 more (3 cans with 2 tennis balls in each can). 

5 (initial tennis balls) + 6 (new tennis balls) = 11

Ravi now has 11 tennis balls.


In [4]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Ravi has now, we need to follow these steps:

1. Ravi already has 5 tennis balls.
2. He buys 3 cans of tennis balls, and each can has 2 tennis balls. So, the total number of tennis balls in the cans is 3 * 2 = 6.
3. Now, we add the tennis balls Ravi already had (5) to the tennis balls he bought (6). So, the total number of tennis balls Ravi has now is 5 + 6 = 11.

Therefore, Ravi has 11 tennis balls now.


In [5]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a child psychologist, I would recommend Solution 1: "Create a 'Veggie Face' Plate" as the most sustainable approach. Here's why:

1. **Involves the child in the process**: By letting the child help design their own "Veggie Face" plate, they feel more invested in the meal and are more likely to try the vegetables that make up the face.

2. **Encourages creativity and problem-solving**: The process of creating a "Veggie Face" plate requires creativity and problem-solving, which are essential skills for children to develop. This approach helps build confidence and self-esteem.

3. **Makes mealtime more engaging**: A "Veggie Face" plate is a fun and visually appealing presentation that makes mealtime more engaging and exciting for the child. This approach helps create a positive association with vegetables.

4. **No bribing or manipulation**: Unlike the "Stealthy Sneak" method, which involves blending vegetables into a smoothie, this approach doesn'

In [7]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
Introducing "Echoes of Eternity": In this gripping time-travel thriller, a brilliant and reclusive physicist, Emma, discovers a way to manipulate the fabric of time using a revolutionary device known as the "Chrono-Heart." As she travels through the ages, she finds herself drawn to a mysterious and charismatic stranger, Julian, whom she meets across different eras - from the roaring twenties to a dystopian future. But when Emma's actions in the past begin to alter the course of history, she realizes that she's not alone in her journey. A dark and twisted version of her future self, driven by a thirst for revenge and fueled by the secrets of her family's dark past, begins to stalk her across time, threatening to destroy the very fabric of reality and the love that Emma has found with Julian. As the stakes grow higher, Emma must navigate the complexities of time travel, confront her own demons, and make impossible choices to prevent a catastrophic f